In [3]:
from bs4 import BeautifulSoup
import csv, time, json, re, requests
import threading, subprocess, time
import pandas as pd

COVENTRY PAPERS SCRAPPING

In [4]:
# Collecting Coventry publications page

def scrapPapers(start_page = 1, page_limit = 900):

    page = start_page
    url = f"https://pureportal.coventry.ac.uk/en/publications/?format=&page={page}"
    papers = []
    while page < page_limit:
        try:
            pageSource = requests.get(url).text
            soup = BeautifulSoup(pageSource, "html.parser")   
            paperLists = soup.select(".list-result-item")
            if len(paperLists) == 0:
                break
            for paper in paperLists:
                paperInfo = {}

                paperInfo['link'] = paper.select_one('h3.title a')['href']
                paperInfo['title'] = paper.select_one('h3.title a').text

                journal = paper.select_one('a', attrs = {'rel' : 'Journal'})
                paperInfo['journal'] = journal.text
                paperInfo['journalLink'] = journal['href']

                cols = ['date', 'volume', 'pages', 'numberofpages', 'type_classification']

                for x in cols:
                    try:
                        paperInfo[x] = paper.select_one(f'span.{x}').text

                        if x == 'numberofpages':
                            paperInfo[x] = int(paperInfo[x][:-2])
                        elif x == 'pages':
                            paperInfo[x] = paperInfo[x][3:]
                        elif x == 'volume':
                            paperInfo[x] = int(paperInfo[x])

                    except:
                        pass
                
                papers.append(paperInfo)
            print(f"Completed {page} ")
            
            page += 1
            url = f"https://pureportal.coventry.ac.uk/en/publications/?format=&page={page}"

        except: 
            break

    return papers

def getInformationOfDocuments(paperInfo):
    
    source = requests.get(paperInfo['link']).text
    paperSoup = BeautifulSoup(source, "html.parser")
    
    if paperSoup.select_one("div.doi a") is not None:
        paperInfo['doi'] = paperSoup.select_one("div.doi a")['href']
    
    persons = paperSoup.select_one("p.relations.persons")
    
    if persons is not None:
        paperInfo['authors'] = list(map(
                lambda x : x.strip(), 
                persons.text.split(','))
            )
    
    paperInfo['tags'] = [span.text for span in 
            paperSoup.select("li.userdefined-keyword")]

    paperInfo['coventryAuthors'] = [a['href'] for a in 
            persons.select('a', attrs = { 'rel' : 'Person'})]
    
    abstract = paperSoup.select_one(".rendering_researchoutput_abstractportal")

    paperInfo['abstract'] = None

    if abstract:
        paperInfo['abstract'] = abstract.text
    
    # return paperInfo

In [5]:
papers = scrapPapers(start_page = 1)
len(papers)

Completed 1 
Completed 2 
Completed 3 
Completed 4 
Completed 5 
Completed 6 
Completed 7 
Completed 8 
Completed 9 
Completed 10 
Completed 11 
Completed 12 
Completed 13 
Completed 14 
Completed 15 
Completed 16 
Completed 17 
Completed 18 
Completed 19 
Completed 20 
Completed 21 
Completed 22 
Completed 23 
Completed 24 
Completed 25 
Completed 26 
Completed 27 
Completed 28 
Completed 29 
Completed 30 
Completed 31 
Completed 32 
Completed 33 
Completed 34 
Completed 35 
Completed 36 
Completed 37 
Completed 38 
Completed 39 
Completed 40 
Completed 41 
Completed 42 
Completed 43 
Completed 44 
Completed 45 
Completed 46 
Completed 47 
Completed 48 
Completed 49 
Completed 50 
Completed 51 
Completed 52 
Completed 53 
Completed 54 
Completed 55 
Completed 56 
Completed 57 
Completed 58 
Completed 59 
Completed 60 
Completed 61 
Completed 62 
Completed 63 
Completed 64 
Completed 65 
Completed 66 
Completed 67 
Completed 68 
Completed 69 
Completed 70 
Completed 71 
Completed 72 
C

Completed 555 
Completed 556 
Completed 557 
Completed 558 
Completed 559 
Completed 560 
Completed 561 
Completed 562 
Completed 563 
Completed 564 
Completed 565 
Completed 566 
Completed 567 
Completed 568 
Completed 569 
Completed 570 
Completed 571 
Completed 572 
Completed 573 
Completed 574 
Completed 575 
Completed 576 
Completed 577 
Completed 578 
Completed 579 
Completed 580 
Completed 581 
Completed 582 
Completed 583 
Completed 584 
Completed 585 
Completed 586 
Completed 587 
Completed 588 
Completed 589 
Completed 590 
Completed 591 
Completed 592 
Completed 593 
Completed 594 
Completed 595 
Completed 596 
Completed 597 
Completed 598 
Completed 599 
Completed 600 
Completed 601 
Completed 602 
Completed 603 
Completed 604 
Completed 605 
Completed 606 
Completed 607 
Completed 608 
Completed 609 
Completed 610 
Completed 611 
Completed 612 
Completed 613 
Completed 614 
Completed 615 
Completed 616 
Completed 617 
Completed 618 
Completed 619 
Completed 620 
Completed 

34191

In [6]:
# Saving the contents
with open("Papers.json", "w") as f:
    f.write(json.dumps(papers))

In [7]:
# reading the contents of a JSON file
with open("Papers.json", "r") as f:
    papers = json.loads(f.read())
print(f"Loaded {len(papers)} papers")

Loaded 34191 papers


In [8]:
getInformationOfDocuments(papers[0])
papers[0]

{'link': 'https://pureportal.coventry.ac.uk/en/publications/association-of-minimally-processed-and-ultra-processed-food-daily',
 'title': 'Association of minimally processed and ultra-processed food daily consumption with obesity in overweight adults:  a cross-sectional study',
 'journal': 'Association of minimally processed and ultra-processed food daily consumption with obesity in overweight adults:  a cross-sectional study',
 'journalLink': 'https://pureportal.coventry.ac.uk/en/publications/association-of-minimally-processed-and-ultra-processed-food-daily',
 'date': '15 Feb 2023',
 'volume': '(In-Press)',
 'numberofpages': 30,
 'type_classification': 'Article',
 'doi': 'https://doi.org/10.20960/nh.04270',
 'authors': ['Leonardo V Silva',
  'Pedro Pugliesi Abdalla',
  'Lucimere Bohn',
  'Rafael Gavassa de Araújo',
  'Daniel de Freitas Batalhão',
  'Ana Cláudia Rossini Venturini',
  'Anderson Dos Santos Carvalho',
  'Michael Duncan',
  'Jorge Mota',
  'Dalmo Roberto Lopes Machado'],
 

In [9]:
# scrapes papers in parallel using multiple threads
num_threads = 20
lock = threading.Lock()
last = time.time()

def scrapePapersParallel(papers, start, end):
    '''
        in the range [start, end)
    '''

    last = time.time()

    for i in range(start, end):
        try:
            getInformationOfDocuments(papers[i])

            if i % 100 == 0:
                print(f"Completed {i}")
        except:
            print(f"Failed Index {i}")

        try:
            now = time.time()

            # write to file here synchronize against threads
            if threading.current_thread().name == "Thread-0":
                if (now - last) >= 60:
                    lock.acquire()
                    last = now

                    with open("./Papers.json", "w") as f:
                        f.write(json.dumps(papers))
                    
                    print(f"Wrote to file")
                    lock.release()
        except:
            print("Error writing to file")
            
    print(f"Completed thread {threading.current_thread().name}")


In [10]:
# a parallel processing approach to scraping papers using multiple threads
blockSizes = len(papers) // num_threads

startInds = [i * blockSizes for i in range(0, num_threads)]
startInds.append(len(papers))
print(startInds)

threads = []

for ind in range(num_threads):
    new = threading.Thread(
        target=scrapePapersParallel,
        args=(papers, startInds[ind], startInds[ind + 1]),
        name=f"Thread-{ind}"
    )

    threads.append(new)
    new.start()

over_threads = iter(threads)
curr_th = next(over_threads)

while True:
    curr_th.join()
    if curr_th.is_alive():
        continue
    try:
        curr_th = next(over_threads)
    except StopIteration:
        break

[0, 1709, 3418, 5127, 6836, 8545, 10254, 11963, 13672, 15381, 17090, 18799, 20508, 22217, 23926, 25635, 27344, 29053, 30762, 32471, 34191]
Completed 0
Completed 18800
Completed 17100
Completed 15400
Completed 13700
Completed 32500
Wrote to file
Completed 30800
Completed 12000
Completed 29100
Completed 10300
Completed 8600
Completed 27400
Completed 6900
Completed 25700
Wrote to file
Completed 24000
Completed 5200
Completed 22300
Completed 3500
Completed 20600
Completed 1800
Wrote to file
Completed 18900
Completed 100
Completed 17200
Completed 15500
Completed 32600
Completed 13800
Wrote to file
Completed 30900
Completed 12100
Completed 29200
Completed 10400
Completed 27500
Completed 8700
Completed 25800
Wrote to file
Completed 7000
Completed 24100
Completed 5300
Completed 22400
Completed 3600
Completed 20700
Completed 1900
Wrote to file
Completed 19000
Completed 200
Completed 17300
Completed 15600
Completed 32700
Completed 13900
Wrote to file
Completed 31000
Completed 12200
Completed 293

In [13]:
# Reading paper from the JSON file
with open("Papers.json", "r") as f:
    papers = json.loads(f.read())

papers[50]

{'link': 'https://pureportal.coventry.ac.uk/en/publications/challenging-visuality-in-the-time-of-the-anthropocene-fragmentati',
 'title': "Challenging Visuality in the Time of the Anthropocene: Fragmentation and Plurality in Nathalie Miebach's Harvey Twitter SOS",
 'journal': "Challenging Visuality in the Time of the Anthropocene: Fragmentation and Plurality in Nathalie Miebach's Harvey Twitter SOS",
 'journalLink': 'https://pureportal.coventry.ac.uk/en/publications/challenging-visuality-in-the-time-of-the-anthropocene-fragmentati',
 'date': 'May 2023',
 'volume': '(In-Press)',
 'pages': '(In-Press)',
 'type_classification': 'Article',
 'authors': ['Phaedra Shanbaum'],
 'tags': [],
 'coventryAuthors': ['https://pureportal.coventry.ac.uk/en/persons/phaedra-shanbaum'],
 'abstract': None}

In [11]:
papersDf = pd.read_json("Papers.json")
# papersDf

# export as csv
papersDf.to_csv("Papers.csv", index=False)

Collecting Author Profiles

In [12]:
def getProfileURLorNone(url):

    if "no-content" in url:
        return None
    
    pattern = r"^(\/[^?]+)"
    path_match = re.match(pattern, url)
    path = None

    if path_match:
        path = path_match.group(1)

    return path    

def scrapeAuthors(start_page = 1, page_limit = 50):

    page = start_page
    url = f"https://pureportal.coventry.ac.uk/en/persons/?format=&page={page}"

    authors = []

    while page < page_limit:

        try:
            pageSource = requests.get(url).text

            soup = BeautifulSoup(pageSource, "html.parser")
            
            authorList = soup.select("li.grid-result-item div.result-container")

            if len(authorList) == 0:
                break

            for author in authorList:

                try:
                    authorInfo = {}

                    authorInfo['picUrl'] = getProfileURLorNone(
                            author.select_one("img")['src']
                    )

                    if authorInfo['picUrl'] is not None:
                        authorInfo['picUrl'] = 'https://pureportal.coventry.ac.uk/' + authorInfo['picUrl'] 

                    name = author.select_one("a", attrs = { 'rel' : 'Person'})

                    authorInfo['name'] = name.text
                    authorInfo['profileLink'] = name['href']

                    dept = author.select_one(".relations.organisations a", 
                            attrs = { 'rel' : 'Organisation'})

                    authorInfo['department'] = dept.text
                    authorInfo['deptLink'] = dept['href']
                    
                    authors.append(authorInfo)
                except:
                    pass

            print(f"Completed {page} ")
            
            page += 1
            url = f"https://pureportal.coventry.ac.uk/en/persons/?format=&page={page}"
        except: 
            break

    return authors

In [14]:
authors = scrapeAuthors()

print(f"Scraped {len(authors)} authors")

Completed 1 
Completed 2 
Completed 3 
Completed 4 
Completed 5 
Completed 6 
Completed 7 
Completed 8 
Completed 9 
Completed 10 
Completed 11 
Completed 12 
Completed 13 
Completed 14 
Completed 15 
Completed 16 
Completed 17 
Completed 18 
Completed 19 
Completed 20 
Completed 21 
Completed 22 
Completed 23 
Completed 24 
Completed 25 
Completed 26 
Completed 27 
Completed 28 
Completed 29 
Completed 30 
Completed 31 
Completed 32 
Completed 33 
Completed 34 
Completed 35 
Completed 36 
Completed 37 
Completed 38 
Completed 39 
Completed 40 
Completed 41 
Completed 42 
Completed 43 
Completed 44 
Completed 45 
Completed 46 
Scraped 2030 authors


In [15]:
with open("Authors.json", "w") as f:
    f.write(json.dumps(authors))

In [16]:
authorsDf = pd.read_json("Authors.json")
# papersDf

# export as csv
authorsDf.to_csv("Authors.csv", index=False)